In [3]:
!pip install transformers torch numpy tdqm pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.7 MB/s eta 0:00:00
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=1d6f044ad2abc867437551ea05aab1c682520662c38a985d5444419e2e4862e9
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device, n_gpu)

cuda 1


In [6]:

def copy_to_drive(dst):
  !cp 'checkpoint-best-acc/model.bin' dst

In [7]:
import torch.nn as nn


class Model(nn.Module):
    def __init__(self, encoder,config,tokenizer,args):
        super(Model, self).__init__()
        self.encoder = encoder
        self.config=config
        self.tokenizer=tokenizer
        self.args=args

    def forward(self, input_ids=None,labels=None):
        logits=self.encoder(input_ids,attention_mask=input_ids.ne(1))[0]
        prob=torch.softmax(logits,-1)
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-1)
            loss = loss_fct(logits,labels)
            return loss,prob
        else:
            return prob

In [8]:
import json
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset

class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 input_tokens,
                 input_ids,
                 label,

    ):
        self.input_tokens = input_tokens
        self.input_ids = input_ids
        self.label=label

def convert_examples_to_features(js, tokenizer):
    code=' '.join(js['code'].split())
    code_tokens = tokenizer.tokenize(code)[:block_size-2]
    source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
    source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
    padding_length = block_size - len(source_ids)
    source_ids+=[tokenizer.pad_token_id]*padding_length
    return InputFeatures(source_tokens,source_ids,js['label'])

class TextDataset(Dataset):
    def __init__(self, tokenizer, file_path=None):
        self.examples = []
        with open(file_path) as f:
            for line in f:
                js=json.loads(line.strip())
                self.examples.append(convert_examples_to_features(js,tokenizer))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        return torch.tensor(self.examples[i].input_ids),torch.tensor(self.examples[i].label)



In [9]:
print(torch.__version__)
print(torch.cuda.is_available())
# Create a tensor on the CPU
x = torch.tensor([1, 2, 3])

# Move the tensor to the GPU
x = x.to('cuda')

# Check the device of the tensor
print(x.device)



2.0.1+cu118
True
cuda:0


In [18]:
### args
import pandas as pd
import torch
from tqdm import tqdm, trange
import random
import os
import numpy as np
import logging
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
g_res = None
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
seed = 42
model_name_or_path = "microsoft/codebert-base"
tokenizer_name = "microsoft/codebert-base"
train_data_file = "/content/drive/My Drive/ECI/2023-1/dataset/train.jsonl"
eval_data_file = "/content/drive/My Drive/ECI/2023-1/dataset/typescript-test.jsonl"
test_data_file = "/content/drive/My Drive/ECI/2023-1/dataset/testA.jsonl"

base_output_dir = "."
block_size = 256
num_train_epochs = 1
adam_epsilon = 1e-8
learning_rate = 2e-5
weight_decay = 0.0
train_batch_size = 2
max_grad_norm = 1.0
eval_batch_size = 16
weight_decay = 0

def test(model, tokenizer):
    # Note that DistributedSampler samples randomly
    eval_dataset = TextDataset(tokenizer, test_data_file)
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=eval_batch_size)

    # Eval!
    logger.info("***** Running Test *****")
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()
    logits=[]
    labels=[]
    for batch in tqdm(eval_dataloader,total=len(eval_dataloader)):
        inputs = batch[0].to(device)
        label=batch[1].to(device)
        with torch.no_grad():
            logit = model(inputs)
            logits.append(logit.cpu().numpy())
            labels.append(label.cpu().numpy())

    logits=np.concatenate(logits,0)
    labels=np.concatenate(labels,0)
    preds=logits.argmax(-1)
    with open(os.path.join(base_output_dir,"predictions.txt"),'w') as f:
        for example,pred in zip(eval_dataset.examples,preds):
            f.write(str(pred)+ '\n')


def evaluate(model, tokenizer, eval_f):
    eval_output_dir = base_output_dir

    eval_dataset = TextDataset(tokenizer, eval_f)

    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=eval_batch_size,num_workers=4,pin_memory=True)

    # Eval!
    logger.info("***** Running evaluation *****")
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()
    logits=[]
    labels=[]
    for batch in eval_dataloader:
        inputs = batch[0].to(device)
        label=batch[1].to(device)
        with torch.no_grad():
            lm_loss,logit = model(inputs,label)
            eval_loss += lm_loss.mean().item()
            logits.append(logit.cpu().numpy())
            labels.append(label.cpu().numpy())
        nb_eval_steps += 1
    logits=np.concatenate(logits,0)
    labels=np.concatenate(labels,0)
    preds=logits.argmax(-1)
    eval_acc=np.mean(labels==preds)

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.tensor(eval_loss)

    result = {
        "eval_loss": float(perplexity),
        "eval_acc":round(eval_acc,4),
        "labels" : labels,
        "preds": preds
    }
    return result

def train(train_dataset, model, tokenizer, save_as_model_name):
    """ Train the model """
    train_sampler = RandomSampler(train_dataset)

    train_dataloader = DataLoader(train_dataset, sampler=train_sampler,
                                  batch_size=train_batch_size,num_workers=4,pin_memory=True)
    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    max_steps = len(train_dataloader) * num_train_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=max_steps*0.1,
                                                num_training_steps=max_steps)

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", num_train_epochs)
    logger.info("  batch size = %d", train_batch_size)
    logger.info("  Total optimization steps = %d", max_steps)
    best_acc=0.0
    model.zero_grad()


def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYHTONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

def run_evaluate(model_name, load_model_name, eval_f, predname):
  # Set seed
  set_seed(seed)

  config = RobertaConfig.from_pretrained(model_name)
  config.num_labels=15
  tokenizer = RobertaTokenizer.from_pretrained(model_name)
  model = RobertaForSequenceClassification.from_pretrained(model_name, config=config)

  model=Model(model,config,tokenizer,{})
  if load_model_name != None and load_model_name != "":
    print("loading model")
    model.load_state_dict(torch.load(f"drive/My Drive/models/{load_model_name}.bin"))
  model.to(device)
  print(model)
  # Check the device of model parameters
  result=evaluate(model, tokenizer, eval_f)
  logger.info("***** Eval results *****")
  for key in sorted(result.keys()):
      try:
        logger.info("  %s = %s", key, str(round(result[key],4)))
      except:
        continue


  res = result

  ID_TO_LABEL = {
      0: 'awsservice',
      1: 'serverless',
      2: 'object-storage',
      3: 'microservices',
      4: 'event-driven',
      5: 'nosql-storage',
      6: 'iot',
      7: 'batch-processing',
      8: 'big-data',
      9: 'data-warehouse',
      10: 'streaming',
      11: 'data-orch'
  }


  s = np.array((res["preds"]))
  s = np.transpose(s)
  df = pd.DataFrame(s, columns = ['preds'])


  count_by_label = df.groupby('preds').agg({'preds': [("total","count")]})
  count_by_label.reset_index()

  #count_by_label['name'] = count_by_label['preds'].map(ID_TO_LABEL)



  return count_by_label



def run_training(model_name, tokenizer_name, save_as_model_name):
  # Set seed
  set_seed(seed)

  config = RobertaConfig.from_pretrained(model_name)
  config.num_labels=15
  tokenizer = RobertaTokenizer.from_pretrained(tokenizer_name)
  model = RobertaForSequenceClassification.from_pretrained(model_name,config=config)

  model=Model(model,config,tokenizer,{})

      # multi-gpu training (should be after apex fp16 initialization)
  model.to(device)
  print(model)
  # Check the device of model parameters
  print(next(model.parameters()).device)
  torch.cuda.get_device_name(0)
  train_dataset = TextDataset(tokenizer, train_data_file)
  train(train_dataset, model, tokenizer, save_as_model_name)


In [11]:

def run_testing():
  # Set seed
  set_seed(seed)



  config = RobertaConfig.from_pretrained(model_name_or_path)
  config.num_labels=15
  tokenizer = RobertaTokenizer.from_pretrained(tokenizer_name)
  model = RobertaForSequenceClassification.from_pretrained(model_name_or_path,config=config)

  model=Model(model,config,tokenizer,{})

  model.load_state_dict(torch.load('drive/My Drive/models/codebert.bin'))
  model.to(device)
  print(model)
  # Check the device of model parameters
  test(model, tokenizer)





In [38]:
#run_training("microsoft/unixcoder-base", "microsoft/unixcoder-base", "unixcoder")
#run_training("Salesforce/codet5-base", "Salesforce/codet5-base", "codet52")
#run_training("roberta-base", "roberta-base", "roberta")

models = [("microsoft/codebert-base", "codebert"), ("microsoft/unixcoder-base", "unixcoder"), ("roberta-base", "roberta"), ("Salesforce/codet5-base", "codet5")]
trained_res = {}
for mld in models:
  base_name, nick = mld
  trained_res[nick] = {}
  for ll in ["go", "java", "python", "typescript"]:
    print("running", base_name, ll)
    trained_res[nick][ll] = run_evaluate(base_name, "", f"/content/drive/My Drive/ECI/2023-1/dataset/{ll}-test.jsonl", "p-pred")




running microsoft/codebert-base go


Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be 

Model(
  (encoder): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear

INFO:__main__:***** Eval results *****
INFO:__main__:  eval_acc = 0.0
INFO:__main__:  eval_loss = 2.595


running microsoft/codebert-base java


...

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:__main__:***** Running evaluation *****
INFO:__main__:  Num examples = 375
INFO:__main__:  Batch size = 16
INFO:__main__:***** Eval results *****
INFO:__main__:  eval_acc = 0.0053
INFO:__main__:  eval_loss = 2.7639


running roberta-base typescript


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
You should pr

Model(
  (encoder): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:__main__:***** Running evaluation *****
INFO:__main__:  Num examples = 2491
INFO:__main__:  Batch size = 16
INFO:__main__:***** Eval results *****
INFO:__main__:  eval_acc = 0.0012
INFO:__main__:  eval_loss = 2.7746


In [51]:
trained_res["unixcoder"]["python"]

,preds
,total
preds,
0,31
1,8
2,76
3,31
4,8
5,7
6,10
8,31


In [37]:
r.

preds
0     90
1     41
2     77
3     61
4     75
5     19
6      1
7      3
9      2
10     6
Name: preds, dtype: int64

Results

CODEbert GPU
Typescript eval
```
INFO:__main__:***** Running evaluation *****
INFO:__main__:  Num examples = 2491
INFO:__main__:  Batch size = 16
INFO:__main__:***** Eval results *****
INFO:__main__:  eval_acc = 0.9695
INFO:__main__:  eval_loss = 0.2735
```

Python eval
```
INFO:__main__:***** Running evaluation *****
INFO:__main__:  Num examples = 375
INFO:__main__:  Batch size = 16
INFO:__main__:***** Eval results *****
INFO:__main__:  eval_acc = 0.6053
INFO:__main__:  eval_loss = 2.4102
```
Java eval
```
INFO:__main__:***** Running evaluation *****
INFO:__main__:  Num examples = 75
INFO:__main__:  Batch size = 16
INFO:__main__:***** Eval results *****
INFO:__main__:  eval_acc = 0.16
INFO:__main__:  eval_loss = 9.1889
```
Go eval
```
INFO:__main__:***** Eval results *****
INFO:__main__:  eval_acc = 0.0
INFO:__main__:  eval_loss = 11.0979
```

UNIX Code

Typescript
```
0.98
01
```
Python
```
0.84
1.3
```
Java
```
0.13
10
```
Go
```
0.0
11
```


CodeT5
Typescript
```
0.94
01
```
Python
```
0.75
1.5
```
Java
```
0.13
8.3
```
Go
```
0.5
4.5
```


Roberta

Typescript
```
0.97
0.15
```
Python
```
0.65
2.2
```
Java
```
0.14
9.0
```
Go
```
0.0
10
```




